In [ ]:
path = '/Users/hunfen/OneDrive/General Files/ゼミー/20201110/2020-11-04/Topography005.sxm'
from Nanonis import read_file
f = read_file(path)

In [ ]:
f.raw_header

In [ ]:
f.header

In [ ]:
f.num_channels

In [ ]:
f.data